# Imports

In [3]:
import numpy as np
import pandas as pd
from pandas_datareader import data as web
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline
import datetime as dt
import mplfinance as mpf

# Function that Saves Stock Data to CSV

# Function that Returns a Dataframe from a CSV

# Add Daily Return to Dataframe

# Returns Total Return over Time

# Matplotlib Finance

# Simpole Price Plot